In [36]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 13: Care Package ---</h2><p>As you ponder the solitude of space and the ever-increasing three-hour roundtrip for messages between you and Earth, you notice that the Space Mail Indicator Light is blinking.  To help keep you sane, the Elves have sent you a care package.</p>
<p>It's a new game for the ship's <a href="https://en.wikipedia.org/wiki/Arcade_cabinet">arcade cabinet</a>! Unfortunately, the arcade is <em>all the way</em> on the other end of the ship. Surely, it won't be hard to build your own - the care package even comes with schematics.</p>
<p>The arcade cabinet runs <a href="9">Intcode</a> software like the game the Elves sent (your puzzle input). It has a primitive screen capable of drawing square <em>tiles</em> on a grid.  The software draws tiles to the screen with output instructions: every three output instructions specify the <code>x</code> position (distance from the left), <code>y</code> position (distance from the top), and <code>tile id</code>. The <code>tile id</code> is interpreted as follows:</p>
<ul>
<li><code>0</code> is an <em>empty</em> tile.  No game object appears in this tile.</li>
<li><code>1</code> is a <em>wall</em> tile.  Walls are indestructible barriers.</li>
<li><code>2</code> is a <em>block</em> tile.  Blocks can be broken by the ball.</li>
<li><code>3</code> is a <em>horizontal paddle</em> tile.  The paddle is indestructible.</li>
<li><code>4</code> is a <em>ball</em> tile.  The ball moves diagonally and bounces off objects.</li>
</ul>
<p>For example, a sequence of output values like <code>1,2,3,6,5,4</code> would draw a <em>horizontal paddle</em> tile (<code>1</code> tile from the left and <code>2</code> tiles from the top) and a <em>ball</em> tile (<code>6</code> tiles from the left and <code>5</code> tiles from the top).</p>
<p>Start the game. <em>How many block tiles are on the screen when the game exits?</em></p>
</article>


In [37]:
from copy import deepcopy
from itertools import batched
from math import inf
from typing import Generator

import matplotlib.animation as animation


class IntcodeComputer:
    def __init__(self, s: str) -> None:
        self.state = {a: int(i) for a, i in enumerate(s.split(","))}
        self.ip = 0

    def run(self) -> Generator[int | None, int, None]:
        offset = 0

        while True:
            mode_c, mode_b, mode_a, opcode = self.opcode()

            if opcode == 1:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = c + b
            elif opcode == 2:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = c * b
            elif opcode == 3:
                f = yield
                self.state[self.adress(mode_c, offset)] = f
            elif opcode == 4:
                c = self.value(mode_c, offset)
                yield c
            elif opcode == 5:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                if c != 0:
                    self.ip = b
            elif opcode == 6:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                if c == 0:
                    self.ip = b
            elif opcode == 7:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = 1 if c < b else 0
            elif opcode == 8:
                c = self.value(mode_c, offset)
                b = self.value(mode_b, offset)
                self.state[self.adress(mode_a, offset)] = 1 if c == b else 0
            elif opcode == 9:
                offset += self.value(mode_c, offset)
            elif opcode == 99:
                return
            else:
                raise ValueError("1202 program alarm")

    def opcode(self):
        code = self.state[self.ip]
        self.ip += 1
        opcode = code % 100
        code //= 100
        mode_c = code % 10
        code //= 10
        mode_b = code % 10
        code //= 10
        mode_a = code % 10
        return mode_c, mode_b, mode_a, opcode

    def value(self, mode: int, offset: int) -> int:
        adress = self.adress(mode, offset)
        if adress > -1:
            if adress not in self.state:
                self.state[adress] = 0
            return self.state[adress]

        raise ValueError(
            f"Adress should be bigger then 0: adress={self.state[self.ip] + offset}"
        )

    def adress(self, mode: int, offset: int) -> int:
        ip = self.ip
        self.ip += 1

        if mode == 0:
            return self.state[ip]

        if mode == 1:
            return ip

        if mode == 2:
            return self.state[ip] + offset

        raise ValueError(f"Unknown adress mode: mode={mode}")

    def play(self, generate_video=False) -> int:
        dp = {}
        scores = []

        x_min, x_max = inf, 0
        y_min, y_max = inf, 0

        self.state[0] = 2
        gen = self.run()

        ball_x, paddle_x = 0, 0
        for x, y, b in batched(gen, 3):
            if x == -1 and y == 0:
                scores.append(b)
                break

            x_min, x_max = min(x, x_min), max(x, x_max)
            y_min, y_max = min(y, y_min), max(y, y_max)
            dp[(x, y)] = b
            if b == 3:  # paddle
                paddle_x = x
            elif b == 4:  # ball
                ball_x = x

        display = [
            [dp[(x, y)] for x in range(x_min, x_max + 1)]
            for y in range(y_min, y_max + 1)
        ]

        ims = []
        fig, ax = None, None

        if generate_video:
            fig, ax = plt.subplots()
            im = ax.imshow(deepcopy(display), cmap="seismic", animated=True)
            ax.imshow(deepcopy(display), cmap="seismic")
            ims.append([im])
        try:
            i = 0
            while True:
                if ball_x < paddle_x:
                    joystick = -1
                elif ball_x > paddle_x:
                    joystick = 1
                else:
                    joystick = 0

                x = None
                while x is None:
                    x = gen.send(joystick)
                y = gen.send(joystick)
                b = gen.send(joystick)

                if x == -1 and y == 0:
                    scores.append(b)
                else:
                    dp[(x, y)] = b
                    display[y - y_min][x - x_min] = b
                    if b == 3:  # paddle
                        paddle_x = x
                    elif b == 4:  # ball
                        ball_x = x

                    if generate_video:
                        if i % 4 == 3:
                            im = ax.imshow(
                                deepcopy(display), cmap="seismic", animated=True
                            )
                            ims.append([im])

                i += 1

        except StopIteration:
            pass

        if generate_video:
            im = ax.imshow(deepcopy(display), cmap="seismic", animated=True)
            ims.append([im])
            ani = animation.ArtistAnimation(
                fig, ims, interval=50, blit=True, repeat_delay=1000
            )
            ani.save("../output/game.mp4")
            plt.show()

        return scores[-1]

    def __repr__(self) -> str:
        return ",".join(str(i) for i in self.state)


with open("../input/day13.txt") as f:
    puzzle = f.read()

print(
    f"Part 1: {sum(1 for tile in batched(IntcodeComputer(puzzle).run(), 3) if tile[-1] == 2)}"
)

Part 1: 363


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>363</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The game didn't run because you didn't put in any quarters. Unfortunately, you did not bring any <span title="You do have crew quarters, but they won't fit in the machine.">quarters</span>. Memory address <code>0</code> represents the number of quarters that have been inserted; set it to <code>2</code> to play for free.</p>
<p>The arcade cabinet has a <a href="https://en.wikipedia.org/wiki/Joystick">joystick</a> that can move left and right.  The software reads the position of the joystick with input instructions:</p>
<ul>
<li>If the joystick is in the <em>neutral position</em>, provide <code>0</code>.</li>
<li>If the joystick is <em>tilted to the left</em>, provide <code>-1</code>.</li>
<li>If the joystick is <em>tilted to the right</em>, provide <code>1</code>.</li>
</ul>
<p>The arcade cabinet also has a <a href="https://en.wikipedia.org/wiki/Display_device#Segment_displays">segment display</a> capable of showing a single number that represents the player's current score. When three output instructions specify <code>X=-1, Y=0</code>, the third output instruction is not a tile; the value instead specifies the new score to show in the segment display.  For example, a sequence of output values like <code>-1,0,12345</code> would show <code>12345</code> as the player's current score.</p>
<p>Beat the game by breaking all the blocks. <em>What is your score after the last block is broken?</em></p>
</article>

</main>


In [38]:
print(f"Part 2: {IntcodeComputer(puzzle).play()}")
# Generate video with does cost between 4 and 5 minutes
# IntcodeComputer(puzzle).play(True)

Part 2: 17159


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>17159</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

<video width="680" height="680" controls>
  <source src="../output/game.mp4" type="video/mp4">
</video>

<br/>

</main>

> Nice a IntcodeComputer is a fully fledged simulator!
